In [21]:
import sys
import gzip
from pathlib import Path
import pandas as pd

src_dir = Path.cwd().parent

# sys.path strictly for importing modules
sys.path.append(str(src_dir))
from utils.data_utils import *

HOSP_DIR = src_dir / "data" / "mimic-iv" / "hosp"

In [22]:
diabetic_patients = load_data(src_dir / "data" / "processed" / "diabetic_patients.csv.gz")
diabetic_patients.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000635,20642640,7,E119,10
1,10000980,20897796,5,E1122,10
2,10001176,23334588,2,25000,9
3,10001843,21728396,9,E119,10
4,10001877,21320596,6,25000,9


In [23]:
diabetic_ids = diabetic_patients["subject_id"].unique()
diabetic_ids

array(['10000635', '10000980', '10001176', ..., '19999287', '19999379',
       '19999828'], dtype=object)

In [24]:
# Define all file paths for cohort extraction
patients_path = HOSP_DIR / "patients.csv.gz"
admissions_path = HOSP_DIR / "admissions.csv.gz"
d_labitems_path = HOSP_DIR / "d_labitems.csv.gz"
labevents_path = HOSP_DIR / "labevents.csv.gz"
prescriptions_path = HOSP_DIR / "prescriptions.csv.gz"
diagnoses_icd_path = HOSP_DIR / "diagnoses_icd.csv.gz"
procedures_icd_path = HOSP_DIR / "procedures_icd.csv.gz"

In [25]:
# Load data
patients = load_data(patients_path)
admissions = load_data(admissions_path)
diagnoses_icd = load_data(diagnoses_icd_path)
procedures_icd = load_data(procedures_icd_path)

pres_usecols = ["subject_id", "hadm_id", "drug", "starttime", "stoptime"]
prescriptions = load_data(prescriptions_path, usecols=pres_usecols)

# Filter data for diabetic patients
patients = filter_to_cohort(patients, diabetic_ids)
admissions = filter_to_cohort(admissions, diabetic_ids)
diagnoses_icd = filter_to_cohort(diagnoses_icd, diabetic_ids)
procedures_icd = filter_to_cohort(procedures_icd, diabetic_ids)
prescriptions = filter_to_cohort(prescriptions, diabetic_ids)

In [26]:
d_labitems = load_data(d_labitems_path)
diabetes_lab_keywords = ["glucose", "a1c", "hemoglobin a1c", "creatinine", "sodium", "potassium", "bicarbonate"]

target_items = d_labitems[d_labitems['label'].str.lower().str.contains("|".join(diabetes_lab_keywords), na=False)]
target_itemids = target_items['itemid'].astype(str).unique()
print(f"Found {len(target_itemids)} relevant itemids")

Found 71 relevant itemids


In [27]:
# Check overlap between target itemids and labevents
sample_chunk = pd.read_csv(labevents_path, compression='gzip', nrows=100000)
sample_chunk['itemid'] = sample_chunk['itemid'].astype(str)
print("Unique itemids in sample chunk:", sample_chunk['itemid'].nunique())
print("Number of target_itemids present in sample chunk:", sum(sample_chunk['itemid'].isin(target_itemids)))

Unique itemids in sample chunk: 515
Number of target_itemids present in sample chunk: 14397


In [28]:
chunksize = 500_000

chunks = []
for chunk in pd.read_csv(labevents_path, compression='gzip', chunksize=chunksize):
    chunk['subject_id'] = chunk['subject_id'].astype(str)
    chunk['itemid'] = chunk['itemid'].astype(str)

    chunk = chunk[chunk['subject_id'].isin(diabetic_patients['subject_id'])]
    chunk = chunk[chunk['itemid'].isin(target_itemids)]

    if not chunk.empty:
        chunks.append(chunk)

labs = pd.concat(chunks, ignore_index=True)
print(labs.shape)

(8233657, 16)


In [29]:
labs.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,2654,10000635,NaN,89716142,51478,P38GHN,2136-04-08 16:27:00,2136-04-08 22:50:00,NaN,NaN,mg/dL,NaN,NaN,NaN,ROUTINE,NEG.
1,2691,10000635,NaN,32171451,50882,NaN,2138-09-29 15:10:00,2138-09-29 16:07:00,27,27.0,mEq/L,22.0,32.0,NaN,STAT,NaN
2,2693,10000635,NaN,32171451,50912,NaN,2138-09-29 15:10:00,2138-09-29 16:07:00,1.2,1.2,mg/dL,0.4,1.1,abnormal,STAT,NaN
3,2695,10000635,NaN,32171451,50931,NaN,2138-09-29 15:10:00,2138-09-29 16:07:00,___,108.0,mg/dL,70.0,100.0,abnormal,STAT,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
4,2696,10000635,NaN,32171451,50971,NaN,2138-09-29 15:10:00,2138-09-29 16:07:00,3.6,3.6,mEq/L,3.3,5.1,NaN,STAT,NaN


In [30]:
labs["valuenum"].value_counts()

valuenum
139.0     167465
138.0     162167
140.0     160974
25.0      147557
137.0     146266
           ...  
1922.4         1
2436.1         1
1130.0         1
7921.3         1
1184.8         1
Name: count, Length: 8222, dtype: int64

In [31]:
labs['chartdate'] = pd.to_datetime(labs['charttime'], errors='coerce').dt.date
labs['valuenum'] = pd.to_numeric(labs['valuenum'], errors='coerce')

daily_labs = (
    labs.groupby(['subject_id', 'chartdate', 'itemid'])['valuenum']
    .mean()
    .unstack('itemid')
)

print(daily_labs.shape)
daily_labs.head()

(1372192, 51)


itemid                 50803  50809  50822  50824  50837  50841  50842  50847  \
subject_id chartdate                                                            
10000635   2136-04-08    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
           2138-09-29    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
           2141-08-15    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
           2142-12-23    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
           2143-06-06    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

itemid                 50848  50852  ...  51100  51106  51478  51790  51981  \
subject_id chartdate                 ...                                      
10000635   2136-04-08    NaN    NaN  ...    NaN    NaN    NaN    NaN    NaN   
           2138-09-29    NaN    NaN  ...    NaN    NaN    NaN    NaN    NaN   
           2141-08-15    NaN    NaN  ...    NaN    NaN    NaN    NaN    NaN   
           2142-12-23    NaN    NaN  ...    NaN    NaN    NaN    NaN    NaN   
           2143-06-06    NaN    NaN  ...    NaN    NaN    NaN    NaN    NaN   

itemid                 52024  52546  52569  52610  52623  
subject_id chartdate                                      
10000635   2136-04-08    NaN    NaN    NaN    NaN    NaN  
           2138-09-29    NaN    NaN    NaN    NaN    NaN  
           2141-08-15    NaN    NaN    NaN    NaN    NaN  
           2142-12-23    NaN    NaN    NaN    NaN    NaN  
           2143-06-06    NaN    NaN    NaN    NaN    NaN  

[5 rows x 51 columns]

In [32]:
def create_condition_flag(diagnoses_icd, code_prefixes, flag_name):
    matched = diagnoses_icd[
        diagnoses_icd["icd_code"].str.startswith(tuple(code_prefixes))
    ]

    flag = (
        matched.groupby("subject_id")
        .size()
        .gt(0)
        .rename(flag_name)
        .reset_index()
    )
    return flag


In [33]:
patients_summary = patients[["subject_id", "gender", "anchor_age"]].rename(columns={"anchor_age": "age"})
admissions_summary = (
    admissions.groupby("subject_id")
    .agg(
        n_admissions=("hadm_id", "nunique"),
        first_admission_date=("admittime", "min"),
        last_admission_date=("admittime", "max"),
    )
    .reset_index()
)
HTN_CODES = ["I10", "401"]
CKD_CODES = ["N18", "585"]
OBESITY_CODES = ["E66", "2780"]
NEUROPATHY_CODES = ["E104", "E114", "G60", "G62", "2506", "3572"]
RETINOPATHY_CODES = ["E103", "E113", "H35", "3620"]
HEART_DISEASE_CODES = ["I2", "I42", "I50", "41", "425", "428"]

htn_flag = create_condition_flag(diagnoses_icd, HTN_CODES, "hypertension_flag")
ckd_flag = create_condition_flag(diagnoses_icd, CKD_CODES, "ckd_flag")
obesity_flag = create_condition_flag(diagnoses_icd, OBESITY_CODES, "obesity_flag")
neuropathy_flag = create_condition_flag(diagnoses_icd, NEUROPATHY_CODES, "neuropathy_flag")
retinopathy_flag = create_condition_flag(diagnoses_icd, RETINOPATHY_CODES, "retinopathy_flag")
heart_flag = create_condition_flag(diagnoses_icd, HEART_DISEASE_CODES, "heart_disease_flag")

prescriptions["is_insulin"] = prescriptions["drug"].str.contains("insulin", case=False)

insulin_flag = (
    prescriptions.groupby("subject_id")["is_insulin"]
    .any()
    .reset_index()
    .rename(columns={"is_insulin": "insulin_flag"})
)

obesity_flag["obesity_flag"].value_counts()

obesity_flag
True    11842
Name: count, dtype: int64

In [34]:
cohort = (
    patients_summary
    .merge(admissions_summary, on="subject_id", how="left")
    .merge(htn_flag, on="subject_id", how="left")
    .merge(ckd_flag, on="subject_id", how="left")
    .merge(obesity_flag, on="subject_id", how="left")
    .merge(neuropathy_flag, on="subject_id", how="left")
    .merge(retinopathy_flag, on="subject_id", how="left")
    .merge(heart_flag, on="subject_id", how="left")
    .merge(insulin_flag, on="subject_id", how="left")
)

cohort.to_csv("../data/processed/diabetic_cohort_summary.csv.gz", index=False, compression="gzip")

cohort.head()

,subject_id,gender,age,n_admissions,first_admission_date,last_admission_date,hypertension_flag,ckd_flag,obesity_flag,neuropathy_flag,retinopathy_flag,heart_disease_flag,insulin_flag
0,10000635,F,74,2,2136-06-19 14:24:00,2143-12-23 14:55:00,True,NaN,True,NaN,NaN,NaN,False
1,10000980,F,73,7,2188-01-03 17:41:00,2193-08-15 01:01:00,NaN,True,True,True,True,True,True
2,10001176,F,64,1,2186-11-29 03:56:00,2186-11-29 03:56:00,True,NaN,NaN,NaN,NaN,True,True
3,10001843,M,73,2,2131-11-09 16:05:00,2134-12-05 00:10:00,NaN,True,True,NaN,NaN,True,True
4,10001877,M,89,2,2149-05-21 15:53:00,2150-11-21 23:02:00,True,NaN,NaN,NaN,NaN,True,True


In [35]:
cohort["subject_id"] = cohort["subject_id"].astype(str)

daily_labs = daily_labs.reset_index()
patient_day_table = daily_labs.merge(
    cohort,
    on="subject_id",
    how="left"
)

print(patient_day_table.shape)
patient_day_table.head()

(1372192, 65)


,subject_id,chartdate,50803,50809,50822,50824,50837,50841,50842,50847,...,n_admissions,first_admission_date,last_admission_date,hypertension_flag,ckd_flag,obesity_flag,neuropathy_flag,retinopathy_flag,heart_disease_flag,insulin_flag
0,10000635,2136-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2136-06-19 14:24:00,2143-12-23 14:55:00,True,NaN,True,NaN,NaN,NaN,False
1,10000635,2138-09-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2136-06-19 14:24:00,2143-12-23 14:55:00,True,NaN,True,NaN,NaN,NaN,False
2,10000635,2141-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2136-06-19 14:24:00,2143-12-23 14:55:00,True,NaN,True,NaN,NaN,NaN,False
3,10000635,2142-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2136-06-19 14:24:00,2143-12-23 14:55:00,True,NaN,True,NaN,NaN,NaN,False
4,10000635,2143-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2136-06-19 14:24:00,2143-12-23 14:55:00,True,NaN,True,NaN,NaN,NaN,False


In [37]:
patient_day_table.to_csv("../data/processed/diabetic_patient_day_table.csv.gz", compression="gzip", index=False)